# K-Nearest Neighbors

Determinar o tipo de vidro a partir de suas propriedades físico-quimicas

Dataset obtido no Kaggle (https://www.kaggle.com/uciml/glass)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dados = pd.read_csv('glass.csv')

In [ ]:
dados.head()

RI -> indice de refração

Na, Mg, Al, Si, K, Ca, Ba Fe -> quantidade de sódio, magnésio, alumínio, silicone, potássio, cálcio, bário e ferro

Type -> tipo de vidro

Verificando os tipos de vidros presentes na amostra

In [ ]:
dados['Type'].value_counts()

In [ ]:
classes = dados['Type'].unique()
n_classes = len(classes)

In [ ]:
sns.countplot(x='Type',data=dados)

Verificando a distribuição de cada elemento

In [ ]:
dados['Na'].plot(kind='hist',title='Na')

In [ ]:
dados['Mg'].plot(kind='hist',title='Mg')

In [ ]:
dados['Al'].plot(kind='hist',title='Al')

In [ ]:
dados['Si'].plot(kind='hist',title='Si')

In [ ]:
dados['K'].plot(kind='hist',title='K')

In [ ]:
dados['Ca'].plot(kind='hist',title='Ca')

In [ ]:
dados['Ba'].plot(kind='hist',title='Ba')

In [ ]:
dados['Fe'].plot(kind='hist',title='Fe')

Verificando distribuição do índice de refração

In [ ]:
dados['RI'].plot(kind='hist',title='RI')

Verificando a exitência de NaNs e nulos

In [ ]:
dados.isna().sum()

In [ ]:
dados.isnull().sum()

Verifica-se que não existem NaNs ou nulos

Observando os valores de cada variável, notamos que o processo de normalização não parece ser necessário. 

Separando as variáveis X e Y

In [ ]:
X = dados.drop('Type',axis=1).values
Y = dados['Type'].values

In [ ]:
from sklearn.preprocessing import label_binarize

In [ ]:
Y = label_binarize(Y, classes=classes)

Criando amostra de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_treino,X_teste,Y_treino,Y_teste=train_test_split(X,Y,test_size=0.25,random_state=0)

Aplicando modelo KNN

No modelo KNN o número de vizinhos é uma variável de entrada. Para melhor determinar este número, iremos cacular a acurácia no intervalo entre 1 e 10.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
vizinhos = []
acc_treino = []
acc_teste = []
std = []

In [ ]:
for i in range(1,11):
    vizinhos.append(i)
    knn=KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_treino,Y_treino)
    acc_treino.append(knn.score(X_treino,Y_treino))
    acc_teste.append(knn.score(X_teste,Y_teste))
    Y_previsto = knn.predict(X_teste)
    std.append(np.std(Y_previsto==Y_teste)/np.sqrt(Y_previsto.shape[0]))

Graficando acurácias em função do número de vizinhos

In [ ]:
plt.scatter(vizinhos,acc_treino,label='Amostra de treino')
plt.fill_between(range(1,11),np.array(acc_teste)-1*np.array(std),np.array(acc_teste)+1*np.array(std), alpha=0.40)
plt.scatter(vizinhos,acc_teste,label='Amostra de teste')
plt.xlabel('Número de vizinhos')
plt.ylabel('Acurácia')
plt.xlim([0,11])
plt.legend()
plt.tight_layout()

Observando as acurácias, notamos que k=1 tem o melhor valor, porém, o mesmo estaria superestimando o modelo. Desta forma, iremos adora k=6 por ter a segunda melhor acurácia na amostra de teste

In [ ]:
knn=KNeighborsClassifier(n_neighbors=6)

In [ ]:
knn.fit(X_treino,Y_treino)

In [ ]:
Y_previsto = knn.predict(X_teste)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm=confusion_matrix(Y_teste.argmax(axis=1),Y_previsto.argmax(axis=1))

In [ ]:
cm

In [ ]:
from sklearn.metrics import classification_report,f1_score,precision_score,average_precision_score,recall_score,accuracy_score

Relatório de classificação

In [ ]:
cr = classification_report(Y_teste,Y_previsto,labels=[0,1,2,3])
print(cr)

F1-score

In [ ]:
f1_macro = f1_score(Y_teste,Y_previsto,average='macro')
print("F1 Macro score = {:0.2f}%".format(f1_macro*100))

f1_wei = f1_score(Y_teste,Y_previsto,average='weighted')
print("F1 Weighted score = {:0.2f}%".format(f1_wei*100))

Precision score

In [ ]:
precisao_macro = precision_score(Y_teste,Y_previsto,average='macro')
print("Precision Macro score = {:0.2f}%".format(precisao_macro*100))

precisao_wei = precision_score(Y_teste,Y_previsto,average='weighted')
print("Precision Weighted score = {:0.2f}%".format(precisao_wei*100))

Recall score

In [ ]:
rec_macro = recall_score(Y_teste,Y_previsto,average='macro')
print("Recall score Macro = {:0.2f}%".format(rec_macro*100))

rec_wei = recall_score(Y_teste,Y_previsto,average='weighted')
print("Recall score Weighted = {:0.2f}%".format(rec_wei*100))

Accuracy score

In [ ]:
acc = accuracy_score(Y_teste,Y_previsto)
print("Accuracy score = {:0.2f}%".format(acc*100))

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score,auc
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_teste[:, i], Y_previsto[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])*100

In [ ]:
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i],label=str(i)+' AUC='+str(round(roc_auc[i],2))+"%")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

In [ ]:
from sklearn.metrics import precision_recall_curve,auc

In [ ]:
lr_precision = dict()
lr_recall = dict()
lr_auc = dict()
for i in range(n_classes):
    lr_precision[i], lr_recall[i], _ = precision_recall_curve(Y_teste[:, i], Y_previsto[:, i])
    lr_auc[i] = auc(lr_recall[i], lr_precision[i])*100

In [ ]:
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i],label=str(i)+' AUC='+str(round(lr_auc[i],2))+"%")
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()